In [13]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [14]:
import FinanceLib as fl

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
#activate logger
today_dt = fl.date.today().strftime('%Y%m%d')    
log_dir = fl.LOG_DIR + '/logs/' + today_dt

dt_to = fl.date.today()
dt_from = fl.date.today() + fl.datetime.timedelta(days=-2)

if not os.path.exists(log_dir):
    os.makedirs(log_dir)
fl.logging.basicConfig(filename=log_dir + '/app.log', filemode='a+', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%H:%M:%S', level=fl.logging.DEBUG)
logger = fl.logging.getLogger()

In [ ]:
#load stock list
fl.save_sp500_tickers()

In [4]:
# os.environ['PG_PWD']

In [5]:
#check what load in work
fl.save_stock_quote_to_db("AAPL", "2023-01-01", "2023-06-11")

[*********************100%***********************]  1 of 1 completed


0

In [7]:
#activate logger
today_dt = fl.date.today().strftime('%Y%m%d')    
log_dir = fl.LOG_DIR + '/logs/' + today_dt

dt_to = fl.date.today()
dt_from = fl.date.today() + fl.datetime.timedelta(days=-2)

if not os.path.exists(log_dir):
    os.makedirs(log_dir)
fl.logging.basicConfig(filename=log_dir + '/app.log', filemode='a+', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%H:%M:%S', level=fl.logging.DEBUG)
logger = fl.logging.getLogger()

In [ ]:
fl.get_all_stock_by_period("2020-01-01", "2023-01-01", logger)

C:\Projects\FinanceProject\FinanceLib.py:219: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


SWKS
[*********************100%***********************]  1 of 1 completed
SEDG
[*********************100%***********************]  1 of 1 completed
SBUX
[*********************100%***********************]  1 of 1 completed
STLD
[*********************100%***********************]  1 of 1 completed
SNPS
[*********************100%***********************]  1 of 1 completed
TMUS
[*********************100%***********************]  1 of 1 completed
TROW
[*********************100%***********************]  1 of 1 completed
TTWO
[*********************100%***********************]  1 of 1 completed
TER
[*********************100%***********************]  1 of 1 completed
TSLA
[*********************100%***********************]  1 of 1 completed
TXN
[*********************100%***********************]  1 of 1 completed
TSCO
[*********************100%***********************]  1 of 1 completed
TRMB
[*********************100%***********************]  1 of 1 completed
CSX
[*********************100%**********

[*********************100%***********************]  1 of 1 completed
HSIC
[*********************100%***********************]  1 of 1 completed
HOLX
[*********************100%***********************]  1 of 1 completed
HON
[*********************100%***********************]  1 of 1 completed
HST
[*********************100%***********************]  1 of 1 completed
HBAN
[*********************100%***********************]  1 of 1 completed
IDXX
[*********************100%***********************]  1 of 1 completed
ILMN
[*********************100%***********************]  1 of 1 completed
PODD
[*********************100%***********************]  1 of 1 completed
INTC
[*********************100%***********************]  1 of 1 completed
INTU
[*********************100%***********************]  1 of 1 completed
ISRG
[*********************100%***********************]  1 of 1 completed
JBHT
[*********************100%***********************]  1 of 1 completed
JKHY
[*********************100%**************

In [ ]:
fl.daily_update_quote

In [6]:
import yfinance as yf
res = yf.download("AAPL", '2023-06-20', '2023-06-22')

[*********************100%***********************]  1 of 1 completed


In [16]:
fl.get_all_stock_recomendation(logger)

C:\Projects\FinanceProject\FinanceLib.py:219: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


SWKS
SEDG
SBUX
STLD
SNPS
TMUS
TROW
TTWO
TER
TSLA
TXN
TSCO
TRMB
CSX
ULTA
UAL
VRSN
VRSK
VRTX
VTRS
WBA
WBD
WTW
WYNN
XEL
ZBRA
ZION
ATVI
ADBE
ADP
AKAM
ALGN
LNT
GOOGL
GOOG
AMZN
AAL
AEP
AMGN
ADI
ANSS
APA
AAPL
AMAT
ACGL
ADSK
AXON
BKR
TECH
BIIB
BKNG
AVGO
CHRW
CDNS
CZR
CDW
CHTR
CINF
CTAS
CSCO
CME
CTSH
CMCSA
CEG
CPRT
AUDCAD
AUDCHF
CSGP
COST
XRAY
DXCM
FANG
DLTR
EBAY
EA
ENPH
EQIX
ETSY
EVRG
NVDA
NXPI
ORLY
ODFL
ON
PCAR
PANW
PARA
PAYX
PYPL
PEP
POOL
PFG
EXPE
EXPD
FFIV
FAST
FITB
FSLR
FTNT
FOXA
FOX
GEHC
GEN
GILD
HAS
HSIC
HOLX
HON
HST
HBAN
IDXX
ILMN
PODD
INTC
INTU
ISRG
JBHT
JKHY
KDP
KLAC
KHC
LRCX
LKQ
MKTX
MAR
MTCH
BRK.B
PRU
TDG
TRV
TFC
TYL
AUDCNH
AMD
TSN
BF.B
USB
UDR
CI
UNP
UPS
URI
UNH
UHS
VLO
VZ
VFC
VICI
V
VMC
WAB
WMT
WM
WAT
WEC
WFC
WELL
WST
ALLE
ALL
MO
AMCR
AEE
AXP
AIG
AMT
AWK
AMP
ABC
AME
APH
AON
APTV
ANET
AJG
AIZ
T
ATO
AZO
BR
BRO
BG
CPT
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CE
CNC
CNP
CDAY
CF
CRL
SCHW
CVX
AUDCZK
AUDDKK
CMG
CB
CHD
C
CFG
CLX
CMS
KO
CL
CMA
CAG
COP
ED
STZ
DOV
DOW
DTE
DUK
DD
DXC
EMN
E

{'State': 3, 'CntError': 0, 'lst_error': []}

In [7]:
res

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-06-20,184.410004,186.100006,184.410004,185.009995,185.009995,49799100
2023-06-21,184.899994,185.410004,182.589996,183.960007,183.960007,49477400


In [11]:
stock = yf.Ticker(f"AAPL=x")
data = stock.history(period="1mo", interval="1d").reset_index()

AAPL=X: No data found, symbol may be delisted


In [9]:
data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2023-05-22 00:00:00+01:00,0.598000,0.598393,0.593700,0.598000,0,0.0,0.0
1,2023-05-23 00:00:00+01:00,0.596800,0.598450,0.595100,0.596800,0,0.0,0.0
2,2023-05-24 00:00:00+01:00,0.596163,0.596030,0.591000,0.596163,0,0.0,0.0
3,2023-05-25 00:00:00+01:00,0.591820,0.592571,0.588450,0.591820,0,0.0,0.0
4,2023-05-26 00:00:00+01:00,0.588080,0.590610,0.587700,0.588080,0,0.0,0.0
5,2023-05-29 00:00:00+01:00,0.590600,0.592520,0.590000,0.590600,0,0.0,0.0
6,2023-05-30 00:00:00+01:00,0.590514,0.592525,0.588200,0.590514,0,0.0,0.0
7,2023-05-31 00:00:00+01:00,0.590486,0.591848,0.588728,0.590486,0,0.0,0.0
8,2023-06-01 00:00:00+01:00,0.592030,0.597000,0.590300,0.592030,0,0.0,0.0
9,2023-06-02 00:00:00+01:00,0.595300,0.601557,0.595185,0.595300,0,0.0,0.0
